In [ ]:
import os, sys
import pandas as pd
from sklearn.manifold import MDS
from sklearn.cluster import KMeans

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from techminer2 import *

# import_scopus_file(scopus_file='raw-documents.csv',)
# import_references(scopus_file="raw-references.csv")

directory = "/workspaces/techminer2/data/"
import_scopus_file(directory)

In [ ]:
import pandas as pd

documents = pd.read_csv("documents.csv")
documents.raw_keywords


In [ ]:
documents.raw_index_keywords

In [ ]:
documents.raw_author_keywords

In [ ]:
from techminer import *
directory = "/workspaces/techminer-api/data/"
collaboration_indicators("countries", directory=directory).head()

In [ ]:
import pandas as pd

m = pd.DataFrame(
    {
    'a': [1, 1, 1, 2, 2, 2],
    'b': ['A', 'B', 'D', 'A', 'B', 'C'],
    'v': [1, 2, 3, 4, 5, 6]
    }
)
m.pivot(index='a', columns='b', values='v')

In [ ]:
import pandas as pd

sorted(pd.read_csv("documents.csv").columns)